In [ ]:


import warnings
from sklearn import metrics

###########
###########test predicitons from features only compare models 
#############
with warnings.catch_warnings():
    fps = 1.6
    warnings.simplefilter("ignore")
    n_tracks = resampled_rev.shape[0]
    test_size = 0.3
    n_splits = 10
    results_dict = {}  # Store mean/std r² per i and n_timesteps
    best_models = {}   # Store best model per (i, n_timesteps)
    lstm_lag = 0
    idxs = [[0],[1],[2],[3],[4],[5]]


    results_dict_score_overall = {}
    best_models_score_overall ={}
    results_dict_auc_overall ={}
    best_models_auc_overall = {}
    
    results_dict_score_stim ={}
    best_models_score_stim = {}
    best_models_auc_stim = {}
    
    results_dict_auc_stim={}
    
    t_to_all_data = {}
    t_to_all_models = {}
    t_to_all_stim_scores ={}
    t_to_overall_aucs = {}
    t_to_best_model_using_min_idx_score ={}
    
    n_timesteps_list = []
    t_to_best_model_idx_auc = {}
    t_to_best_model_idx_score = {}
    for n_timesteps in range(2, 15, 4):#range(1, 15, 4):
        print(f"\nProcessing i={i}, n_timesteps={n_timesteps}")

        all_data = []
        n_stim = len(resampled_onsets)
        stim_i_to_score_means = {i:[] for i in range(n_stim)}
        stim_i_to_score_stds = {i:[] for i in range(n_stim)}
        
        stim_i_to_Xs = {}
        stim_i_to_Ys = {}
        
        ##########prep data 
        for i, (resampled_onset, exp_onset) in enumerate(zip(resampled_onsets, exp_onsets)):
            onset = resampled_onset-1
            X_stim_all, feature_names = prep_FB_inputs_features_only(resampled_vel, resampled_acc, resampled_curve, resampled_rev, resampled_turn, onset, n_timesteps, z,  lstm_lag = lstm_lag, include_beh = False)#shape (n_tracks, n_frames, n_features)
            Y_latency = latency_to_reversal(exp_rev, exp_onset, max_latency = 6*durations[i]+1)/6 # nan where not 
            no_rev_at_onset = Y_latency!=0
            Y_prob = np.logical_not(np.isnan(Y_latency[no_rev_at_onset]))# classfy y/n did rev happen
            X_prob = copy.deepcopy(X_stim_all[no_rev_at_onset])
            stim_i_to_Xs[i] = X_prob
            stim_i_to_Ys[i] = Y_prob
       
        all_stim_X_stim_features, all_stim_Y, feature_names_all = combine_FB_features_across_stim(stim_i_to_Xs, stim_i_to_Ys,  feature_names)
            
            
        ###    
        # X_train, X_test, y_train, y_test = train_test_split(all_stim_X_stim_features, all_stim_Y,  test_size=test_size, random_state=random_state)
        overall_scores = []
        overall_aucs = []
        all_stim_aucs = []
        all_stim_scores =[]# np.zeros((n_splits,n_stim))
        
        best_model_idx = []
        models = [] # n_splits
        ######split data and fit model on split 
        for split_idx in range(n_splits):
            X_train, X_test, y_train, y_test = train_test_split(all_stim_X_stim_features, all_stim_Y,  test_size=test_size, random_state=split_idx)
            all_data.append((X_train, X_test, y_train, y_test))
            
            param_grid = {
                'C': [0.001, 0.01, 0.1, 1, 10, 100]
            }

            model = LinearSVC(penalty='l1', dual=False, max_iter=10000)
            # Grid search
            grid = GridSearchCV(model, param_grid, cv=5, scoring='balanced_accuracy')
            grid.fit(X_train, y_train)
            c = grid.best_params_['C']
            best_model = grid.best_estimator_

            #get stim_i for each row in test data
            stim_bin_test = X_test[:, -1*n_stim:]
            stim_i_test = np.argmax(stim_bin_test, axis = 1)
            stim_scores = [] #model specfic for each stim 
            stim_aucs = []
            for j in range(n_stim):
                
                X_test_stim = X_test[stim_i_test == j, :]
                y_test_stim = y_test[stim_i_test == j]
                
                y_pred = best_model.predict(X_test_stim)
                
                
                fpr, tpr, thresholds = metrics.roc_curve(y_test_stim, y_pred, pos_label=2)
                stim_auc = metrics.auc(fpr, tpr)
                stim_score = balanced_accuracy_score(y_test_stim, y_pred)  
                stim_scores.append(stim_score)
                stim_aucs.append(stim_auc)
            all_stim_scores.append(stim_scores)
            all_stim_aucs.append(stim_aucs)
            
            y_pred = best_model.predict(X_test)
            
            fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=2)
            overall_auc = metrics.auc(fpr, tpr)
            overall_score = balanced_accuracy_score(y_test, y_pred) 
            
            overall_scores.append(overall_score)
            overall_aucs.append(overall_auc)
            models.append((best_model, c))
            
            
            # overall_balanced_accuracy_scores.append(tuple(all_scores))
            # overall_aucs.append(tuple(all_aucs))
            # balanced_accuracy_scores.append(all_scores)
            # aucs.append(all_aucs)
            
                
        all_stim_scores = np.array(all_stim_scores)#n_splits,n_stim
        all_stim_aucs = np.array(all_stim_aucs)
        
        
        ###overall scores
        accuracy_mean_overall = np.mean(overall_scores)
        accuracy_std_overall= np.std(overall_scores)
        
        best_model_idx_score_overall = np.argmax(overall_scores)
        # np.argmin(overall_scores)
        best_model_score_overall, _ = models[best_model_idx_score_overall]
        t_to_best_model_idx_score[n_timesteps] = best_model_idx_score_overall
        
        results_dict_score_overall[n_timesteps] = (accuracy_mean_overall, accuracy_std_overall)
        best_models_score_overall[ n_timesteps] = best_model_score_overall

        
        ###overall aucs 
        auc_mean_overall = np.mean(overall_aucs)
        auc_std_overall = np.std(overall_aucs)
        
        best_model_idx_auc_overall = np.argmax(overall_aucs)
        best_model_auc_overall, _ = models[best_model_idx_auc_overall]
        t_to_best_model_idx_auc[n_timesteps] = best_model_idx_auc_overall
        
        results_dict_auc_overall[n_timesteps] = (accuracy_mean_overall, accuracy_std_overall)
        best_models_auc_overall[ n_timesteps] = best_model_score_overall
        t_to_all_models[ n_timesteps] = models

        #####stim scores
        accuracy_mean_stim_specific = np.mean(all_stim_scores, axis =0)
        accuracy_std_stim_specific = np.std(all_stim_scores, axis =0)

        
        best_model_idx_score_stim_min = np.argmax(np.min(all_stim_scores, axis = 1)) # has lowest min on all stim 
        best_model_score_stim_min, _ = models[best_model_idx_score_stim_min]
        t_to_best_model_using_min_idx_score[n_timesteps] = best_model_idx_score_stim_min
        

        results_dict_score_stim[n_timesteps] = (accuracy_mean_stim_specific, accuracy_std_stim_specific)
        best_models_score_stim[ n_timesteps] = best_model_score_stim_min
        
        #####stim aucs
        auc_mean_stim_specific = np.mean(all_stim_aucs, axis = 0)
        auc_std_stim_specific = np.std(all_stim_aucs, axis = 0)
        
        
        best_model_idx_auc_stim_min = np.argmax(np.min(all_stim_aucs, axis = 1)) # has lowest min on all stim 
        best_model_auc_stim_min, _ = models[best_model_idx_auc_stim_min]
        
        
        t_to_all_stim_scores[n_timesteps] = all_stim_scores
        t_to_overall_aucs[n_timesteps] = all_stim_aucs
        
        results_dict_auc_stim[n_timesteps] = (auc_mean_stim_specific, auc_std_stim_specific)
        best_models_auc_stim[ n_timesteps] = best_model_auc_stim_min
        t_to_all_data[n_timesteps] = all_data
        

        n_timesteps_list.append(n_timesteps)

    ####plot overall mean and std overall and for each stim  
    fig, ax = plt.subplots()
    accuracy_mean_overall_over_t =[]
    accuracy_std_overall_over_t = []
    stim_acc_mean_over_t = []
    stim_acc_std_over_t = []
    for n_timesteps in n_timesteps_list:

        accuracy_mean_overall, accuracy_std_overall = results_dict_score_overall[n_timesteps]
        accuracy_mean_overall_over_t.append(accuracy_mean_overall)
        accuracy_std_overall_over_t.append(accuracy_std_overall)
        accuracy_mean_stim_specific, accuracy_std_stim_specific = results_dict_score_stim[n_timesteps]
        stim_acc_mean_over_t.append(accuracy_mean_stim_specific)
        stim_acc_std_over_t.append(accuracy_std_stim_specific)
    stim_acc_mean_over_t = np.array(stim_acc_mean_over_t)
    stim_acc_std_over_t = np.array(stim_acc_std_over_t)
    for j in range(n_stim):
        ax.errorbar(np.array(n_timesteps_list)/fps, stim_acc_mean_over_t[:,j], yerr=stim_acc_std_over_t[:,j], fmt='-o', capsize=5, label = f"stim {j}")#color=cmap1(i),
        
    ax.errorbar(np.array(n_timesteps_list)/fps, accuracy_mean_overall_over_t, yerr=accuracy_std_overall_over_t, #color=cmap1(i),
                fmt='-o', capsize=5, label = f"overall")
    ax.set_title('mean balanced accuracy performance over models for given stim ')
    ax.set_ylabel("balanced accuracy")
    ax.set_xlabel("max time delay (s)")
    ax.legend()
        
    #  ####for best overall mdoel plto how it did overall and on each stim 
    # fig, ax = plt.subplots()
    
    # for j in range(n_stim):
    #     all_scores = []
    #     for n_timesteps in n_timesteps_list:
    #         all_data = t_to_all_data[n_timesteps]
            
    #         best_model_score_overall = best_models_score_overall[ n_timesteps]
       
    #         for X_train, X_test, y_train, y_test in all_data:
    #             stim_bin_test = X_test[:, -1*n_stim:]
    #             stim_i_test = np.argmax(stim_bin_test, axis = 1)
    #             X_test_stim = X_test[stim_i_test == j, :]
    #             y_test_stim = y_test[stim_i_test == j, :]

    #             y_pred = best_model_score_overall.predict(X_test)
    #             score = balanced_accuracy_score(y_test, y_pred) 
    #             all_scores.append(score)
    #     ax.plot(n_timesteps_list, all_scores, label = f"stim {j}")
    # ax.set_xlabel("timestep")
    # ax.set_ylabel("acc")

    # # all_scores = []
    # # for n_timesteps in n_timesteps_list:
    # #     X_train, X_test, y_train, y_test = t_to_all_data[n_timesteps]
            

    # #     best_model_score_overall = best_models_score_overall[ n_timesteps]
    # #     y_pred = best_model_score_overall.predict(X_test)
    # #     score = balanced_accuracy_score(y_test, y_pred) 
    # #     all_scores.append(score)
    # #     ax.plot(n_timesteps_list, all_scores, label = "overall")
    # ax.legend()
    
    ####plot best over stim how it does overall - score for each T overall 
    # fig, ax = plt.subplots()
    
    # for j in range(n_stim):
    #     all_scores = []
    #     for n_timesteps in n_timesteps_list:
    #         X_train, X_test, y_train, y_test = t_to_all_data[n_timesteps]
    #         best_model_score_overall = best_model_score_stim_min[ n_timesteps]
       
        
    #         X_test_stim = X_test[stim_i_test == j, :]
    #         y_test_stim = y_test[stim_i_test == j, :]
            
    #         y_pred = best_model_score_overall.predict(X_test)
    #         score = balanced_accuracy_score(y_test, y_pred) 
    #         all_scores.append(score)
    #     ax.plot(n_timesteps_list, all_scores, label = f"stim {j}")
    # all_scores = []
    # for n_timesteps in n_timesteps_list:
    #     X_train, X_test, y_train, y_test = t_to_all_data[n_timesteps]
            

    #     best_model_score_overall = best_model_score_stim_min[ n_timesteps]
    #     y_pred = best_model_score_overall.predict(X_test)
    #     score = balanced_accuracy_score(y_test, y_pred) 
    #     all_scores.append(score)
    #     ax.plot(n_timesteps_list, all_scores, label = "overall")
    # ax.legend()
    
    ####plot score overall score on each stim (each line a model)
    for n_timesteps in n_timesteps_list:
        all_stim_scores = t_to_all_stim_scores[n_timesteps]
        
        fig, ax = plt.subplots()
        for k in range(n_splits): 
            ax.plot(np.arange(n_stim), all_stim_scores[k])
        ax.set_title(f"balanced accuracy across all models; timestep{n_timesteps}")
        ax.set_xlabel("stim number")
        ax.set_ylabel("accuracy")
        idx = t_to_best_model_idx_score[n_timesteps]
        ax.plot(np.arange(n_stim), all_stim_scores[idx], linewidth = 3, label = "best")
        
        idx = t_to_best_model_using_min_idx_score[n_timesteps]
        ax.plot(np.arange(n_stim), all_stim_scores[idx], linewidth = 3, label = "best using min")
    


In [ ]:
    
def visualize_model_classification(model, model_label, feature_names, X_train, X_test, y_train, y_test, n_time_steps, get_coeff = True,  feature_names_ordered = None,  xlim = None, coeffs = None):
    
    # model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"{model_label}Accuracy:", accuracy_score(y_test, y_pred))
    print(f"{model_label}Precision:", precision_score(y_test, y_pred, average='weighted'))
    print(f"{model_label}Recall:", recall_score(y_test, y_pred, average='weighted'))
    print(f"{model_label}F1 Score:", f1_score(y_test, y_pred, average='weighted'))
    # disp = ConfusionMatrixDisplay.from_estimator(model, X_train,  y_train, cmap='Blues')
    # disp.ax_.set_title(f"{model_label}train")
    disp = ConfusionMatrixDisplay.from_estimator(model, X_test,  y_test, cmap='Blues')
    disp.ax_.set_title(f"{model_label}test")
    if get_coeff:
        fig, ax = plot_coeffs(model, feature_names, model_label, n_time_steps, feature_names_ordered = feature_names_ordered, xlim = xlim, coeffs = coeffs)
        ax.set_title(f"{model_label}")
    return model, y_pred

train_model_feature_list

In [ ]:
####get rev prob; resampled  all beh feature s
feature_list = [resampled_vel, resampled_acc, resampled_curve]
feature_labels = ["vel", "accel", "curv"]
data_prep_fnc = prep_data_rev_prob
model_fit_fnc = fit_eval_linearSVC
result_visualization_fnc = visualize_model_classification
data_onsets = resampled_onsets
title_label = f"{neuron}; Reversal prediction;\n resampled; all beh features; Mean ± STD balanced accuracy"
fps = 1.6

feature_to_best_models  = get_plot_multi_stim_model(
                                                    data_prep_fnc,
                                                    model_fit_fnc, 
                                                    result_visualization_fnc, 
                                                    feature_list, 
                                                    feature_labels,
                                                    z, 
                                                    data_onsets,
                                                    exp_onsets, 
                                                    exp_rev, 
                                                    durations,
                                                    fps,
                                                    # title_label,
                                                    test_size = 0.3, 
                                                    n_splits = 10, 
                                                    lstm_lag = 0, 

                                                    all_timesteps = None, 
                                                    check_warnings = True)


####get rev prob; resampled  all beh feature s
# feature_list = [resampled_vel, resampled_acc, resampled_curve]
# feature_labels = ["vel", "accel", "curv"]
# data_prep_fnc = prep_data_rev_prob
# model_fit_fnc = fit_eval_linearSVC
# result_visualization_fnc = visualize_model_classification
# data_onsets = resampled_onsets
title_label = f"{neuron}; Reversal prediction;\n resampled; single beh features; Mean ± STD balanced accuracy"

best_model_score_stim_min, t_to_best_model_using_min_idx_score, t_to_all_data, t_to_all_feature_names  = get_plot_multi_stim_model_single_feature(
                                                                        data_prep_fnc,
                                                                        model_fit_fnc, 
                                                                        result_visualization_fnc, 
                                                                        feature_list, 
                                                                        feature_labels,
                                                                        z, 
                                                                        data_onsets,
                                                                        exp_onsets, 
                                                                        exp_rev, 
                                                                        durations,
                                                                        fps,
                                                                        # title_label,
                                                                        test_size = 0.3, 
                                                                        n_splits = 10, 
                                                                        lstm_lag = 0, 

                                                                        all_timesteps = None, 
                                                                        check_warnings = True)

In [ ]:
feature_list = [resampled_vel, resampled_acc, resampled_curve]
feature_labels = ["vel", "accel", "curv"]
data_prep_fnc = prep_data_latency
model_fit_fnc = fit_eval_lasso
result_visualization_fnc = visualize_model_regression
data_onsets = resampled_onsets
title_label = f"{neuron}; Reversal latency;\n all features resamppled; Mean ± STD balanced accuracy"
fps = 1.6

feature_to_best_models  = get_plot_multi_stim_model(
                                                    data_prep_fnc,
                                                    model_fit_fnc, 
                                                    result_visualization_fnc, 
                                                    feature_list, 
                                                    feature_labels,
                                                    z, 
                                                    data_onsets,
                                                    exp_onsets, 
                                                    exp_rev, 
                                                    durations,
                                                    fps,
                                                    # title_label,
                                                    test_size = 0.3, 
                                                    n_splits = 10, 
                                                    lstm_lag = 0, 

                                                    all_timesteps = None, 
                                                    check_warnings = True)

title_label = f"{neuron}; Reversal latency;\n single features resampled;  Mean ± STD balanced accuracy"

best_model_score_stim_min, t_to_best_model_using_min_idx_score, t_to_all_data, t_to_all_feature_names  = get_plot_multi_stim_model(
                                                                        data_prep_fnc,
                                                                        model_fit_fnc, 
                                                                        result_visualization_fnc, 
                                                                        feature_list, 
                                                                        feature_labels,
                                                                        z, 
                                                                        data_onsets,
                                                                        exp_onsets, 
                                                                        exp_rev, 
                                                                        durations,
                                                                        fps,
                                                                        # title_label,
                                                                        test_size = 0.3, 
                                                                        n_splits = 10, 
                                                                        lstm_lag = 0, 

                                                                        all_timesteps = None, 
                                                                        check_warnings = True)

In [ ]:
def get_transitions(stateseq):
    ''' copied from keypoint moseq github: keypoint-moseq/keypoint_moseq/util.py'''
    transitions = np.nonzero(stateseq[1:] != stateseq[:-1])[0] + 1
    starts = np.insert(transitions, 0, 0)
    ends = np.append(transitions, len(stateseq))
    return  transitions, starts, ends



    
def get_state_start_ends(state, stateseq,  starts= None, ends = None):
    if starts is None: 
        transitions, starts, ends = get_transitions(stateseq)
    # for state1, state2 in state_pairs:
    state_start_is = np.argwhere(state==stateseq[starts[:-1]]).flatten()
    
    state_starts = starts[state_start_is]
    state_ends = ends[state_start_is] ##check 
    
    return state_starts, state_ends

def get_state_track_start_ends(state, all_stateseqs,  starts= None, ends = None):
    track_start_ends = []
    all_tracks, all_starts, all_ends = [], [], []
    for i, stateseq in enumerate(all_stateseqs):
        if starts is None: 
            transitions, starts, ends = get_transitions(stateseq)
        # for state1, state2 in state_pairs:
        state_start_is = np.argwhere(state==stateseq[starts[:-1]]).flatten()
        tracks = np.ones(state_start_is.shape)*i
        state_starts = starts[state_start_is]
        state_ends = ends[state_start_is] ##check 
        
        all_tracks.append(tracks)
        all_starts.append(state_starts)
        all_ends.append(state_ends)
        
        
    return np.concatenate([
                            np.concatenate(all_tracks)[:, None],
                            np.concatenate(all_starts)[:, None],
                            np.concatenate(all_ends)[:, None]
                            
                            ])


def plot_mean_line_graph_with_individ_traces(data,n_pre_frames,  fps = 6,  fig = None, ax = None, seconds = None,vline = None, title = "", 
                                             xlabel ="seconds post-stim start", 
                                             ylabel ="speed (um/s)", color = "blue", ylim =None,  ylabel_font_size = 8, 
                                             max_n_tracks = 30,line_width=1.0, 
                                             
                                             ):
    if fig is None:# types.NoneType 
        fig, ax = plt.subplots()
    if data.shape[0] ==0:
        return fig, ax
    
    n_tracks, n_frames = data.shape
    if n_tracks>max_n_tracks:
        track_is = np.arange(n_tracks)
        np.random.shuffle(track_is)
        sampled_track_is = track_is[:max_n_tracks]
        sampled_data = data[sampled_track_is,:]
    else:
        sampled_data = data
    average_row = np.nanmean(sampled_data, axis=0)
   
    # Plot each row with grayed lines
    if seconds is None:
        n_frames = sampled_data.shape[1]
        # seconds = np.arange(n_frames*params.n_stim_types)/params.fps
        seconds = np.arange(n_frames)/fps - n_pre_frames/fps
        vline = 0
    for row in sampled_data:
        ax.plot(seconds, row, color='gray', alpha=0.5, linewidth=line_width)

    # Plot the average from BehaviorAnalyzer.PlottingFunctions.get_event_speed_line_plots row with a more prominent line
    ax.plot(seconds, average_row, color=color, label='Average', linewidth=2)

    # Customize labels and legend
    if xlabel is not None: 
        ax.set_xlabel(xlabel)
    if ylabel is not None: 
        ax.set_ylabel(ylabel, fontsize = ylabel_font_size)
    if title is not None: 
        ax.set_title(title)
    if ylim is not None:
        ax.set_ylim(ylim)
    if vline is not None:
        ax.axvline(x=vline, color='black', linestyle='--')

    return fig, ax


# def align_feature_to_rev_start( exp_date_to_beh, exp_date_to_feature, n_pre_frames = 60, max_plotting_length_frames = 500, 
#                                rev_duration_min_frames = None, rev_duration_max_frames = None ,  rev_state = 1
#                                ):
   
#     buffered_event_features = []
#     for exp_date, featureseq in exp_date_to_feature.items():
#         # rsldsseq = date_to_discrete_rslds_states[exp_date]
#         behseq = exp_date_to_beh[exp_date]
#         featureseq = exp_date_to_feature[exp_date]
#         state_starts, state_ends = get_state_start_ends(rev_state , behseq,  starts= None, ends = None)
#         for rev_start_i,rev_end_i in zip( state_starts, state_ends):
#             rev_duration_frames = rev_end_i-rev_start_i
#             if rev_duration_min_frames is not None and rev_duration_frames<rev_duration_min_frames:
#                 continue
#             if rev_duration_max_frames is not None and rev_duration_frames>rev_duration_max_frames:
#                 continue
#             frame_start = max(rev_start_i-n_pre_frames, 0)
#             frame_end = min(rev_start_i+max_plotting_length_frames, rev_end_i)
        
#             n_rev_frames = frame_end-rev_start_i
#             n_fwd_frames = rev_start_i-frame_start
#             buffered_event_feature = np.zeros(n_pre_frames+max_plotting_length_frames)
#             buffered_event_feature[:] = np.nan
#             buffered_event_feature[n_pre_frames+-1*n_fwd_frames:n_pre_frames+n_rev_frames] = featureseq[frame_start:frame_end]
#             buffered_event_features.append(buffered_event_feature[:, None])

#     if len(buffered_event_features)==0: 
#         return np.array([])

#     # buffered_event_features = np.concatenate(buffered_event_features, axis = 0)
#     buffered_event_features = np.concatenate(buffered_event_features, axis = 1)
#     return buffered_event_features



def align_feature_to_rev_start_from_start_end_is(feature_val, event_start_end_is,  n_pre_frames = 60, max_plotting_length_frames = 500, 
                               rev_duration_min_frames = None, rev_duration_max_frames = None ,  rev_state = 1, starts =None, ends = None, 
                            #    earliest_start_frame = None, earliest_end_frame = None
                               
                               ):
    
    buffered_event_features = []
    durations = event_start_end_is[:,2]-event_start_end_is[:,1]
    dur_bool = np.logical_and(durations>rev_duration_min_frames, durations<=rev_duration_max_frames).astype('bool')
    event_start_end_is_duration = event_start_end_is[dur_bool]
    for track, rev_start_i, rev_end_i in event_start_end_is_duration:
        
    # # for i, (featureseq, z_w) in enumerate(zip(feature_val,z)):
    #     # rsldsseq = date_to_discrete_rslds_states[exp_date]
        
    #     # featureseq = exp_date_to_feature[exp_date]
    #     # state_starts, state_ends = get_state_start_ends(
    #     #                                                 rev_state,
    #     #                                                 z_w,  
    #     #                                                 starts= None, 
    #     #                                                 ends = None
    #     #                                                 )
    #     # for rev_start_i,rev_end_i in zip( state_starts, state_ends):
        
    #         rev_duration_frames = rev_end_i-rev_start_i
    #         if rev_duration_frames>=rev_duration_min_frames and rev_duration_frames<rev_duration_max_frames:

            frame_start = max(rev_start_i-n_pre_frames, 0)
            frame_end = min(rev_start_i+max_plotting_length_frames, rev_end_i)
        
            n_rev_frames = frame_end-rev_start_i
            n_fwd_frames = rev_start_i-frame_start
            buffered_event_feature = np.zeros(n_pre_frames+max_plotting_length_frames)
            buffered_event_feature[:] = np.nan
            buffered_event_feature[(n_pre_frames+-1*n_fwd_frames):(n_pre_frames+n_rev_frames)] = feature_val[track, frame_start:frame_end]
            buffered_event_features.append(buffered_event_feature[:, None])

    if len(buffered_event_features)==0: 
        return np.array([])

    # buffered_event_features = np.concatenate(buffered_event_features, axis = 0)
    buffered_event_features = np.concatenate(buffered_event_features, axis = 1)
    return buffered_event_features


def align_feature_to_rev_end_from_start_end_is(feature_val, event_start_end_is,  
                            n_pre_frames = 90, max_plotting_length_frames = 150, 
                            rev_duration_min_frames = None, rev_duration_max_frames = None,  rev_state = 1
    ):

    n_frames = feature_val.shape[1]
    buffered_event_features = []
    durations = event_start_end_is[:,2]-event_start_end_is[:,1]
    dur_bool = np.logical_and(durations>rev_duration_min_frames, durations<=rev_duration_max_frames).astype('bool')
    event_start_end_is_duration = event_start_end_is[dur_bool]
    for track, rev_start_i, rev_end_i in event_start_end_is_duration:
    
    # for i, (featureseq, z_w) in enumerate(zip(feature_val,z)):
       
    #     # rsldsseq = date_to_discrete_rslds_states[exp_date]
    #     # behseq = exp_date_to_beh[exp_date]
    #     # featureseq = exp_date_to_feature[exp_date]
    #     state_starts, state_ends = get_state_start_ends(rev_state , z_w,  starts= None, ends = None)
        # for rev_start_i,rev_end_i in zip( state_starts, state_ends):
        #     rev_duration_frames = rev_end_i-rev_start_i
        #     if rev_duration_min_frames is not None and rev_duration_frames<rev_duration_min_frames:
        #         continue
        #     if rev_duration_max_frames is not None and rev_duration_min_frames>rev_duration_max_frames:
        #         continue
            frame_start = max(0, rev_end_i-n_pre_frames) # which frame to reference in freature mat as start of window to plot  
            frame_end = min(rev_end_i+max_plotting_length_frames, n_frames)# which frame to reference in freature mat as end of window to plot
            frame_window = frame_end-frame_start # full elngth to plot 
            max_window_size = max_plotting_length_frames+n_pre_frames
            if n_frames< rev_end_i+max_plotting_length_frames:
                # n_pre_frames = rev_end_i+max_plotting_length_frames-n_frames
                # window_start = max_window_size-frame_window #+rev_end_i+max_plotting_length_frames-n_frames
                diff = rev_end_i+max_plotting_length_frames-n_frames
                window_start = max_window_size-frame_window
                window_start = 0
            else: 
                window_start = max_window_size-frame_window#n_pre_frames
                diff = 0
                

            buffered_event_feature = np.zeros(max_window_size)
            buffered_event_feature[:] = np.nan
            buffered_event_feature[window_start:n_pre_frames+max_plotting_length_frames-diff] = feature_val[track, frame_start:frame_end]
            buffered_event_features.append(buffered_event_feature[:, None])
                
            
            # n_rev_frames = frame_end-rev_start_i
            # n_fwd_frames = rev_start_i-frame_start
            # buffered_event_feature = np.zeros(n_pre_frames+max_plotting_length_frames)
            # buffered_event_feature[:] = np.nan
            # buffered_event_feature[n_pre_frames+-1*n_fwd_frames:n_pre_frames+n_rev_frames] = featureseq[frame_start:frame_end]
            # buffered_event_features.append(buffered_event_feature[:, None])
  
    if len(buffered_event_features)==0: 
        return np.array([])

    buffered_event_features = np.concatenate(buffered_event_features, axis = 1)
    return buffered_event_features


def align_rev_to_stim_start_from_start_end_is(feature_val, event_start_end_is,  stim_start_frames, 
                            n_pre_frames = 60, max_plotting_length_frames = 500, 
                               rev_duration_min_frames = None, rev_duration_max_frames = None ,  rev_state = 1
                               ):
   
    n_frames = feature_val.shape[1]
    buffered_event_features = []
    durations = event_start_end_is[:,2]-event_start_end_is[:,1]
    dur_bool = np.logical_and(durations>rev_duration_min_frames, durations<=rev_duration_max_frames).astype('bool')
    event_start_end_is_duration = event_start_end_is[dur_bool]
    for track, rev_start_i, rev_end_i in event_start_end_is_duration:
    # for i, (featureseq, z_w) in enumerate(zip(feature_val,z)):
    #     state_starts, state_ends = get_state_start_ends(rev_state , z_w,  starts= None, ends = None)
    #     for rev_start_i,rev_end_i in zip( state_starts, state_ends):
    #         rev_duration_frames = rev_end_i-rev_start_i
    #         if rev_duration_min_frames is not None and rev_duration_frames<rev_duration_min_frames:
    #             continue
    #         if rev_duration_max_frames is not None and rev_duration_frames>rev_duration_max_frames:
    #             continue
            frame_start = max(stim_start_frames-n_pre_frames, 0)
            frame_end = min(stim_start_frames+max_plotting_length_frames, rev_end_i)
        
            n_poststim_frames = frame_end-n_pre_frames
            n_prestim_frames = stim_start_frames-frame_start
            buffered_event_feature = np.zeros(n_pre_frames+max_plotting_length_frames)
            
            buffered_event_feature[:] = np.nan
            buffered_event_feature[(n_pre_frames-n_prestim_frames):n_pre_frames+n_poststim_frames] = feature_val[track, frame_start:frame_end]
            buffered_event_features.append(buffered_event_feature[:, None])

            if len(buffered_event_features)==0: 
                return np.array([])

    # buffered_event_features = np.concatenate(buffered_event_features, axis = 0)
    buffered_event_features = np.concatenate(buffered_event_features, axis = 1)
    return buffered_event_features


def filter_start_ends_is_by_latency(track_start_end_is, stim_start, latency_min, latency_max):
    latencies = track_start_end_is[:, 1]-stim_start
    latency_bool = np.logical_and(latencies>=latency_min, latencies<=latency_max ).astype('bool')
    return track_start_end_is[latency_bool]

'''get track start ends; filter form start being after stim start; do the rest '''

def align_feature_to_rev_start( z, feature_val, n_pre_frames = 60, max_plotting_length_frames = 500, 
                               rev_duration_min_frames = None, rev_duration_max_frames = None ,  rev_state = 1, starts =None, ends = None, 
                            #    earliest_start_frame = None, earliest_end_frame = None
                               
                               ):
    if starts is None: 
        starts, ends = get_state_start_ends(
                                                    rev_state,
                                                    z_w,  
                                                    starts= None, 
                                                    ends = None, 
                                                    # earliest_start_frame = earliest_start_frame, 
                                                    # earliest_end_frame = earliest_end_frame
                                                    
                                                    )
    buffered_event_features = []
    for i, (featureseq, z_w) in enumerate(zip(feature_val,z)):
        # rsldsseq = date_to_discrete_rslds_states[exp_date]
        
        # featureseq = exp_date_to_feature[exp_date]

        for rev_start_i,rev_end_i in zip( starts, ends):
            rev_duration_frames = rev_end_i-rev_start_i
            if rev_duration_min_frames is not None and rev_duration_frames<rev_duration_min_frames:
                continue
            if rev_duration_max_frames is not None and rev_duration_frames>rev_duration_max_frames:
                continue
            frame_start = max(rev_start_i-n_pre_frames, 0)
            frame_end = min(rev_start_i+max_plotting_length_frames, rev_end_i)
        
            n_rev_frames = frame_end-rev_start_i
            n_fwd_frames = rev_start_i-frame_start
            buffered_event_feature = np.zeros(n_pre_frames+max_plotting_length_frames)
            buffered_event_feature[:] = np.nan
            buffered_event_feature[n_pre_frames+-1*n_fwd_frames:n_pre_frames+n_rev_frames] = featureseq[frame_start:frame_end]
            buffered_event_features.append(buffered_event_feature[:, None])

    if len(buffered_event_features)==0: 
        return np.array([])

    # buffered_event_features = np.concatenate(buffered_event_features, axis = 0)
    buffered_event_features = np.concatenate(buffered_event_features, axis = 1)
    return buffered_event_features


# def align_feature_to_rev_end(exp_date_to_beh, exp_date_to_feature,
#                             n_pre_frames = 90, max_plotting_length_frames = 150, 
#                             rev_duration_min_frames = None, rev_duration_max_frames = None,  rev_state = 1
# ):

    
#     buffered_event_features = []    
#     for exp_date, featureseq in exp_date_to_feature.items():
#         n_frames = featureseq.shape[0]
#         # rsldsseq = date_to_discrete_rslds_states[exp_date]
#         behseq = exp_date_to_beh[exp_date]
#         featureseq = exp_date_to_feature[exp_date]
#         state_starts, state_ends = get_state_start_ends(rev_state , behseq,  starts= None, ends = None)
#         for rev_start_i,rev_end_i in zip( state_starts, state_ends):
#             rev_duration_frames = rev_end_i-rev_start_i
#             if rev_duration_min_frames is not None and rev_duration_frames<rev_duration_min_frames:
#                 continue
#             if rev_duration_max_frames is not None and rev_duration_min_frames>rev_duration_max_frames:
#                 continue
#             frame_start = max(0, rev_end_i-n_pre_frames) # which frame to reference in freature mat as start of window to plot  
#             frame_end = min(rev_end_i+max_plotting_length_frames, n_frames)# which frame to reference in freature mat as end of window to plot
#             frame_window = frame_end-frame_start # full elngth to plot 
#             max_window_size = max_plotting_length_frames+n_pre_frames
#             if n_frames< rev_end_i+max_plotting_length_frames:
#                 # n_pre_frames = rev_end_i+max_plotting_length_frames-n_frames
#                 # window_start = max_window_size-frame_window #+rev_end_i+max_plotting_length_frames-n_frames
#                 diff = rev_end_i+max_plotting_length_frames-n_frames
#                 window_start = max_window_size-frame_window
#                 window_start = 0
#             else: 
#                 window_start = max_window_size-frame_window#n_pre_frames
#                 diff = 0
                

#             buffered_event_feature = np.zeros(max_window_size)
#             buffered_event_feature[:] = np.nan
#             buffered_event_feature[window_start:n_pre_frames+max_plotting_length_frames-diff] = featureseq[frame_start:frame_end]
#             buffered_event_features.append(buffered_event_feature[:, None])
                
            
#             # n_rev_frames = frame_end-rev_start_i
#             # n_fwd_frames = rev_start_i-frame_start
#             # buffered_event_feature = np.zeros(n_pre_frames+max_plotting_length_frames)
#             # buffered_event_feature[:] = np.nan
#             # buffered_event_feature[n_pre_frames+-1*n_fwd_frames:n_pre_frames+n_rev_frames] = featureseq[frame_start:frame_end]
#             # buffered_event_features.append(buffered_event_feature[:, None])
  
#     if len(buffered_event_features)==0: 
#         return np.array([])

#     buffered_event_features = np.concatenate(buffered_event_features, axis = 1)
#     return buffered_event_features

def align_feature_to_rev_end( z, feature_val,
                            n_pre_frames = 90, max_plotting_length_frames = 150, 
                            rev_duration_min_frames = None, rev_duration_max_frames = None,  rev_state = 1
    ):

    
    buffered_event_features = []    
    # for exp_date, featureseq in exp_date_to_feature.items():
    for i, (featureseq, z_w) in enumerate(zip(feature_val,z)):
        n_frames = featureseq.shape[0]
        # rsldsseq = date_to_discrete_rslds_states[exp_date]
        # behseq = exp_date_to_beh[exp_date]
        # featureseq = exp_date_to_feature[exp_date]
        state_starts, state_ends = get_state_start_ends(rev_state , z_w,  starts= None, ends = None)
        for rev_start_i,rev_end_i in zip( state_starts, state_ends):
            rev_duration_frames = rev_end_i-rev_start_i
            if rev_duration_min_frames is not None and rev_duration_frames<rev_duration_min_frames:
                continue
            if rev_duration_max_frames is not None and rev_duration_min_frames>rev_duration_max_frames:
                continue
            frame_start = max(0, rev_end_i-n_pre_frames) # which frame to reference in freature mat as start of window to plot  
            frame_end = min(rev_end_i+max_plotting_length_frames, n_frames)# which frame to reference in freature mat as end of window to plot
            frame_window = frame_end-frame_start # full elngth to plot 
            max_window_size = max_plotting_length_frames+n_pre_frames
            if n_frames< rev_end_i+max_plotting_length_frames:
                # n_pre_frames = rev_end_i+max_plotting_length_frames-n_frames
                # window_start = max_window_size-frame_window #+rev_end_i+max_plotting_length_frames-n_frames
                diff = rev_end_i+max_plotting_length_frames-n_frames
                window_start = max_window_size-frame_window
                window_start = 0
            else: 
                window_start = max_window_size-frame_window#n_pre_frames
                diff = 0
                

            buffered_event_feature = np.zeros(max_window_size)
            buffered_event_feature[:] = np.nan
            buffered_event_feature[window_start:n_pre_frames+max_plotting_length_frames-diff] = featureseq[frame_start:frame_end]
            buffered_event_features.append(buffered_event_feature[:, None])
                
            
            # n_rev_frames = frame_end-rev_start_i
            # n_fwd_frames = rev_start_i-frame_start
            # buffered_event_feature = np.zeros(n_pre_frames+max_plotting_length_frames)
            # buffered_event_feature[:] = np.nan
            # buffered_event_feature[n_pre_frames+-1*n_fwd_frames:n_pre_frames+n_rev_frames] = featureseq[frame_start:frame_end]
            # buffered_event_features.append(buffered_event_feature[:, None])
  
    if len(buffered_event_features)==0: 
        return np.array([])

    buffered_event_features = np.concatenate(buffered_event_features, axis = 1)
    return buffered_event_features

def align_rev_to_stim_start(z, feature_val,stim_start_frames, stim_end_frames,
                            n_pre_frames = 60, max_plotting_length_frames = 500, 
                               rev_duration_min_frames = None, rev_duration_max_frames = None ,  rev_state = 1
                               ):
   
    buffered_event_features = []
    # for exp_date, featureseq in exp_date_to_feature.items():
    #     # rsldsseq = date_to_discrete_rslds_states[exp_date]
    #     behseq = exp_date_to_beh[exp_date]
    #     featureseq = exp_date_to_feature[exp_date]
    for i, (featureseq, z_w) in enumerate(zip(feature_val,z)):
        state_starts, state_ends = get_state_start_ends(rev_state , z_w,  starts= None, ends = None)
        for rev_start_i,rev_end_i in zip( state_starts, state_ends):
            rev_duration_frames = rev_end_i-rev_start_i
            if rev_duration_min_frames is not None and rev_duration_frames<rev_duration_min_frames:
                continue
            if rev_duration_max_frames is not None and rev_duration_frames>rev_duration_max_frames:
                continue
            frame_start = max(stim_start_frames-n_pre_frames, 0)
            frame_end = min(stim_start_frames+max_plotting_length_frames, rev_end_i)
        
            n_poststim_frames = frame_end-n_pre_frames
            n_prestim_frames = stim_start_frames-frame_start
            buffered_event_feature = np.zeros(n_pre_frames+max_plotting_length_frames)
            
            buffered_event_feature[:] = np.nan
            buffered_event_feature[(n_pre_frames-n_prestim_frames):n_pre_frames+n_poststim_frames] = featureseq[frame_start:frame_end]
            buffered_event_features.append(buffered_event_feature[:, None])

        if len(buffered_event_features)==0: 
            return np.array([])

    # buffered_event_features = np.concatenate(buffered_event_features, axis = 0)
    buffered_event_features = np.concatenate(buffered_event_features, axis = 1)
    return buffered_event_features

In [ ]:
####get stim algined and rev alogined plots

'''get reversal features'''

# feature_to_dict = {
#     "body angle mag":(exp_date_to_body_angles,  50, lambda x: np.mean(x) ), 
#     "velocity": (exp_date_to_velocity,   50,  lambda x: np.max(np.abs(x))), 
#     "head angle": (exp_date_to_head_angle,  50,  lambda x: np.mean(x)),#np.arange(0, 6, 0.01)), , 
#     "acceleration": (exp_date_to_acceleration, 50, lambda x: np.max(np.abs(x)))
    
    
# }
feature_list = [stim_resampled_vel, stim_resampled_acc, stim_resampled_curve]
feature_labels = ["resampled_vel", "resampled_accel", "resampled_curv"]

'''get rev start aligned '''
n_pre_frames = 16*5
max_plotting_length_frames =  320
dt = 0.06

rev_duration_mins = np.arange(0, 20/dt)[::15]#[-5:]
print(rev_duration_mins)
rev_diff = 0.5/dt
rev_duration_maxs = rev_duration_mins+rev_diff

for rev_duration_min ,rev_duration_max in zip(rev_duration_mins.tolist(),rev_duration_maxs.tolist()):
    
    all_fig, all_axs = plt.subplots(len(features), 1 )
    # for i, feature in enumerate(features): 
    for i, (feature, feature_lbl) in enumerate(zip(feature_list, feature_labels)):
        # exp_date_to_feature, bins, feature_fnc  = feature_to_dict[feature]
        # for rslds_start_state, rslds_end_state in rslds_pairs: 
            # '''get number of reversals that: 
            #     - start in rslds state and end in other rslds state 
            #     - start in rslds state and stay in rslds state 
            #     - start in differnt state and end in rslds state 
            # ...based on this decide which version to plot.. 
            # '''
            # print(feature)
            # # fig, ax = plt.subplots()
            # feature_vals = []
            # beh_states = []
        # buffered_event_features = []
        rev_aligned_feature = align_feature_to_rev_start(resampled_rev, feature, n_pre_frames = n_pre_frames, 
                                                         max_plotting_length_frames =  max_plotting_length_frames, 
                                                         rev_duration_min_frames = rev_duration_min, rev_duration_max_frames = rev_duration_max)
        # ax1 = axs[i]
        # print(rev_aligned_feature.shape)
        # for j in range(2,8):
        fig, ax1 = plt.subplots()
        plot_mean_line_graph_with_individ_traces(rev_aligned_feature.T, fps = 1/dt,  fig = fig, ax =ax1 , seconds = None, vline = n_pre_frames*dt, 
                                                title = f"{np.round(rev_duration_min*dt,2)}", 
                                                xlabel ="seconds post-rev start", 
                                                ylabel =f"{feature_lbl}", color = "blue", ylim =None,  ylabel_font_size = 8, 
                                                max_n_tracks = 30,line_width=1.0, 
                                                
                                                )
        
        
        plot_mean_line_graph_with_individ_traces(rev_aligned_feature.T, fps = 1/dt,  fig = all_fig, ax =all_axs[i] , seconds = None, vline = n_pre_frames*dt, 
                                               title = f"{np.round(rev_duration_min*dt,2)}", 
                                                xlabel ="seconds post-rev start", 
                                                ylabel =f"{feature_lbl}", color = "blue", ylim =None,  ylabel_font_size = 8, 
                                                max_n_tracks = 30,line_width=1.0
                                                
                                                )


                                    
################plto aligned to rev end

# features = ["acceleration", "velocity", "body angle mag",  "head angle"]
max_plotting_length_frames =  120
for rev_duration_min ,rev_duration_max in zip(rev_duration_mins.tolist(),rev_duration_maxs.tolist()):
    # for i, feature in enumerate(features):
    all_fig, axs = plt.subplots(len(features), 1 )
    for i, (feature, feature_lbl) in enumerate(zip(feature_list, feature_labels)):
        # exp_date_to_feature, bins, feature_fnc  = feature_to_dict[feature]

        buffered_event_features = []
        rev_aligned_feature = align_feature_to_rev_end(resampled_rev, feature,
                                n_pre_frames = n_pre_frames, max_plotting_length_frames = max_plotting_length_frames, 
                                rev_duration_min_frames = rev_duration_min, rev_duration_max_frames = rev_duration_max)
        
        plot_mean_line_graph_with_individ_traces(rev_aligned_feature.T, fps = 1/0.06,  fig = all_fig, ax = axs[i], seconds = None,title = "", 
                                                xlabel ="seconds post-rev end", 
                                                ylabel =f"{feature_lbl}", color = "blue", ylim =None,  ylabel_font_size = 8, 
                                                max_n_tracks = 30,line_width=1.0, vline = n_pre_frames*dt)
        
        fig, ax1 = plt.subplots()
        plot_mean_line_graph_with_individ_traces(rev_aligned_feature.T, fps = 1/dt,  fig = fig, ax =ax1 , seconds = None, vline = n_pre_frames*dt, 
                                                    title = f"{np.round(rev_duration_min*dt,2)}", 
                                                    xlabel ="seconds post-rev start", 
                                                    ylabel =f"{feature_lbl}", color = "blue", ylim =None,  ylabel_font_size = 8, 
                                                    max_n_tracks = 30,line_width=1.0, 
                                                    
                                                    )
                                                
################seperate by latency plot aligned to stim staet 

'''get rev start aligned '''
n_pre_frames = 16*5
max_plotting_length_frames =  320
dt = 0.06
fps = 1.6
latency_mins = np.arange(0, int(5*fps))#[::15]#[-5:]
print(latency_mins)
rev_diff = fps
latency_maxs = latency_mins+rev_diff

for resampled_onset in resampled_onsets: 
    stim_start_frames = resampled_onset
    stim_end_frames = resampled_onset+5*fps
    
    for latency_min ,latency_max in zip(latency_mins.tolist(),latency_maxs.tolist()):
        features = [ "acceleration", "velocity", "body angle mag"]
        all_fig, all_axs = plt.subplots(len(features), 1 )
        # for i, feature in enumerate(features): 
        for i, (feature, feature_lbl) in enumerate(zip(feature_list, feature_labels)):
            # exp_date_to_feature, bins, feature_fnc  = feature_to_dict[feature]

            buffered_event_features = []
            stim_aligned_feature = align_rev_to_stim_start(resampled_rev, feature, n_pre_frames =n_pre_frames, 
                                                            max_plotting_length_frames =  max_plotting_length_frames, 
                                                            rev_duration_min_frames = rev_duration_min, rev_duration_max_frames = rev_duration_max)
            # ax1 = axs[i]
            # print(rev_aligned_feature.shape)
            # for j in range(2,8):
            fig, ax1 = plt.subplots()
            plot_mean_line_graph_with_individ_traces(stim_aligned_feature.T, fps = 1/dt,  fig = fig, ax =ax1 , seconds = None, vline = n_pre_frames*dt, 
                                                    title = f"{rev_duration_min*dt}", 
                                                    xlabel ="seconds post-stim start", 
                                                    ylabel =f"{feature_lbl}", color = "blue", ylim =None,  ylabel_font_size = 8, 
                                                    max_n_tracks = 30,line_width=1.0, 
                                                    
                                                    
                                                
                                                    )
            
            
            plot_mean_line_graph_with_individ_traces(stim_aligned_feature.T, fps = 1/dt,  fig = all_fig, ax =all_axs[i] , seconds = None, vline = n_pre_frames*dt, 
                                                    title = f"{rev_duration_min*dt}", 
                                                    xlabel ="seconds post-stim start", 
                                                    ylabel =f"{feature_lbl}", color = "blue", ylim =None,  ylabel_font_size = 8, 
                                                    max_n_tracks = 30,line_width=1.0
                                                    
                                                
                                                )

In [ ]:
####get stim algined and rev alogined plots

'''get reversal features'''

# feature_to_dict = {
#     "body angle mag":(exp_date_to_body_angles,  50, lambda x: np.mean(x) ), 
#     "velocity": (exp_date_to_velocity,   50,  lambda x: np.max(np.abs(x))), 
#     "head angle": (exp_date_to_head_angle,  50,  lambda x: np.mean(x)),#np.arange(0, 6, 0.01)), , 
#     "acceleration": (exp_date_to_acceleration, 50, lambda x: np.max(np.abs(x)))
    
    
# }
feature_list = [resampled_vel, resampled_acc, resampled_curve]
feature_labels = ["flav_vel", "flav_accel", "flav_curv"]

'''get rev start aligned '''
n_pre_frames = 16*5
max_plotting_length_frames =  320
dt = 0.06

rev_duration_mins = np.arange(0, 20/dt)[::15]#[-5:]
print(rev_duration_mins)
rev_diff = 0.5/dt
rev_duration_maxs = rev_duration_mins+rev_diff

for rev_duration_min ,rev_duration_max in zip(rev_duration_mins.tolist(),rev_duration_maxs.tolist()):

    all_fig, all_axs = plt.subplots(len(features), 1 )
    # for i, feature in enumerate(features): 
    for i, (feature, feature_lbl) in enumerate(zip(feature_list, feature_labels)):
        # exp_date_to_feature, bins, feature_fnc  = feature_to_dict[feature]
        # for rslds_start_state, rslds_end_state in rslds_pairs: 
            # '''get number of reversals that: 
            #     - start in rslds state and end in other rslds state 
            #     - start in rslds state and stay in rslds state 
            #     - start in differnt state and end in rslds state 
            # ...based on this decide which version to plot.. 
            # '''
            # print(feature)
            # # fig, ax = plt.subplots()
            # feature_vals = []
            # beh_states = []
        # buffered_event_features = []
        rev_aligned_feature = align_feature_to_rev_start(resampled_rev, feature, n_pre_frames = n_pre_frames, 
                                                         max_plotting_length_frames =  max_plotting_length_frames, 
                                                         rev_duration_min_frames = rev_duration_min, rev_duration_max_frames = rev_duration_max)
        # ax1 = axs[i]
        # print(rev_aligned_feature.shape)
        # for j in range(2,8):
        fig, ax1 = plt.subplots()
        plot_mean_line_graph_with_individ_traces(rev_aligned_feature.T, fps = 1/dt,  fig = fig, ax =ax1 , seconds = None, vline = n_pre_frames*dt, 
                                                title = f"{rev_duration_min*dt}", 
                                                xlabel ="seconds post-rev start", 
                                                ylabel =f"{feature}", color = "blue", ylim =None,  ylabel_font_size = 8, 
                                                max_n_tracks = 30,line_width=1.0, 
                                                
                                                )
        
        
        plot_mean_line_graph_with_individ_traces(rev_aligned_feature.T, fps = 1/dt,  fig = all_fig, ax =all_axs[i] , seconds = None, vline = n_pre_frames*dt, 
                                                title = f"{rev_duration_min*dt}", 
                                                xlabel ="seconds post-rev start", 
                                                ylabel =f"{feature}", color = "blue", ylim =None,  ylabel_font_size = 8, 
                                                max_n_tracks = 30,line_width=1.0
                                                
                                                )

################seperate by latency plot aligned to stim staet 

'''get rev start aligned '''
n_pre_frames = 16*5
max_plotting_length_frames =  320
dt = 0.06
fps = 1.6
latency_mins = np.arange(0, int(5*fps))#[::15]#[-5:]
print(latency_mins)
rev_diff = fps
latency_maxs = latency_mins+rev_diff

for latency_min ,latency_max in zip(latency_mins.tolist(),latency_maxs.tolist()):
    all_fig, all_axs = plt.subplots(len(features), 1 )
    # for i, feature in enumerate(features): 
    for i, (feature, feature_lbl) in enumerate(zip(feature_list, feature_labels)):
        exp_date_to_feature, bins, feature_fnc  = feature_to_dict[feature]

        buffered_event_features = []
        stim_aligned_feature = align_rev_to_stim_start(resampled_rev, feature_val, n_pre_frames =n_pre_frames, 
                                                         max_plotting_length_frames =  max_plotting_length_frames, 
                                                         rev_duration_min_frames = rev_duration_min, rev_duration_max_frames = rev_duration_max)
        # ax1 = axs[i]
        # print(rev_aligned_feature.shape)
        # for j in range(2,8):
        fig, ax1 = plt.subplots()
        plot_mean_line_graph_with_individ_traces(stim_aligned_feature.T, fps = 1/dt,  fig = fig, ax =ax1 , seconds = None, vline = n_pre_frames*dt, 
                                                title = f"{rev_duration_min*dt}", 
                                                xlabel ="seconds post-stim start", 
                                                ylabel =f"{feature}", color = "blue", ylim =None,  ylabel_font_size = 8, 
                                                max_n_tracks = 30,line_width=1.0, 
                                                
                                                
                                            
                                                )
        
        
        plot_mean_line_graph_with_individ_traces(stim_aligned_feature.T, fps = 1/dt,  fig = all_fig, ax =all_axs[i] , seconds = None, vline = n_pre_frames*dt, 
                                                title = f"{rev_duration_min*dt}", 
                                                xlabel ="seconds post-stim start", 
                                                ylabel =f"{feature}", color = "blue", ylim =None,  ylabel_font_size = 8, 
                                                max_n_tracks = 30,line_width=1.0
                                                
                                            
                                                )
        

                                    
################plto aligned to rev end

features = ["acceleration", "velocity", "body angle mag",  "head angle"]
fig, axs = plt.subplots(len(features), 1 )
for i, feature in enumerate(features): 
    exp_date_to_feature, bins, feature_fnc  = feature_to_dict[feature]

    buffered_event_features = []
    rev_aligned_feature = align_feature_to_rev_end(exp_date_to_beh, exp_date_to_feature,
                            n_pre_frames = n_pre_frames, max_plotting_length_frames = max_plotting_length_frames, 
                             rev_duration_min_frames = rev_duration_min, rev_duration_max_frames = rev_duration_max)
    
    plot_mean_line_graph_with_individ_traces(rev_aligned_feature.T, fps = 1/0.06,  fig = fig, ax = axs[i], seconds = None,title = "", 
                                             xlabel ="seconds post-rev end", 
                                             ylabel =f"{feature}", color = "blue", ylim =None,  ylabel_font_size = 8, 
                                             max_n_tracks = 30,line_width=1.0, vline = n_pre_frames*dt)
                                             
                                             

##TODO plot aligned to stim end                            

In [ ]:
def align_rev_to_stim_start_from_start_end_is(feature_val, event_start_end_is,  stim_start_frames, 
                            n_pre_frames = 60, max_plotting_length_frames = 500, 
                               rev_duration_min_frames = -1, rev_duration_max_frames = np.inf ,  rev_state = 1
                               ):
   
    n_frames = feature_val.shape[1]
    buffered_event_features = []
    durations = event_start_end_is[:,2]-event_start_end_is[:,1]
    dur_bool = np.logical_and(durations>rev_duration_min_frames, durations<=rev_duration_max_frames).astype('bool')
    event_start_end_is_duration = event_start_end_is[dur_bool]
    for track, rev_start_i, rev_end_i in event_start_end_is_duration:
    # for i, (featureseq, z_w) in enumerate(zip(feature_val,z)):
    #     state_starts, state_ends = get_state_start_ends(rev_state , z_w,  starts= None, ends = None)
    #     for rev_start_i,rev_end_i in zip( state_starts, state_ends):
    #         rev_duration_frames = rev_end_i-rev_start_i
    #         if rev_duration_min_frames is not None and rev_duration_frames<rev_duration_min_frames:
    #             continue
    #         if rev_duration_max_frames is not None and rev_duration_frames>rev_duration_max_frames:
    #             continue
            print("n_pre_frames", n_pre_frames)
            print("stim_start_frames", stim_start_frames)
            frame_start = max(stim_start_frames-n_pre_frames, 0)
            frame_end = min(stim_start_frames+max_plotting_length_frames, n_frames)
            print("frame_start, frame_end", frame_start, frame_end)
            n_poststim_frames = frame_end-stim_start_frames# n_pre_frames
            n_prestim_frames = stim_start_frames-frame_start
            print("n_prestim_frames", n_prestim_frames, "n_poststim_frames", n_poststim_frames)
            buffered_event_feature = np.zeros(n_pre_frames+max_plotting_length_frames)
            print("n_pre_frames+max_plotting_length_frames", n_pre_frames+max_plotting_length_frames)
            print("n_pre_frames+n_poststim_frames", n_pre_frames+n_poststim_frames )
            print("n_pre_frames-n_prestim_frames", n_pre_frames-n_prestim_frames)
            buffered_event_feature[:] = np.nan
            buffered_event_feature[(n_pre_frames-n_prestim_frames):(n_pre_frames+n_poststim_frames)] = feature_val[track, frame_start:frame_end]
            buffered_event_features.append(buffered_event_feature[:, None])

    if len(buffered_event_features)==0: 
        return np.array([])

    # buffered_event_features = np.concatenate(buffered_event_features, axis = 0)
    buffered_event_features = np.concatenate(buffered_event_features, axis = 1)
    return buffered_event_features

In [ ]:
            og_smooth = smooth_trace(feature.flatten(),  0.5).reshape(stim_resampled_rev.shape[0],-1)
            feature_lbl_og = feature_lbl+"_smooth 0.5"
            
            
            og_smooth_2 = smooth_trace(feature.flatten(),  1.8).reshape(stim_resampled_rev.shape[0],-1)
            feature_lbl_og2 = feature_lbl+"_smooth1.8"
            
            cas_1 = causal_moving_average(feature.flatten(), 1).reshape(stim_resampled_rev.shape[0],-1)
            feature_lbl_caus1 = feature_lbl+"cause_1"
            
                        
            cas_12 = causal_moving_average(feature.flatten(), 2).reshape(stim_resampled_rev.shape[0],-1)
            feature_lbl_caus12 = feature_lbl+"cause_2"
            
            cas_13 = causal_moving_average(feature.flatten(), 3).reshape(stim_resampled_rev.shape[0],-1)
            feature_lbl_caus13 = feature_lbl+"cause_3"
            
            cas_2 = causal_moving_average(feature.flatten(), 5).reshape(stim_resampled_rev.shape[0],-1)
            feature_lbl_caus2 = feature_lbl+"cause_5"
            
            
            
                # smooth_vel = smooth_trace(resampled_vel, 0.5).flatten() 
    # smooth_acc = smooth_trace(resampled_acc, 1.8).flatten()
    # smooth_curve = smooth_trace(resampled_curve, 1).flatten()
    # smooth_vel = causal_moving_average(resampled_vel, 3)
    # smooth_acc = causal_moving_average(resampled_acc, 3)
    # smooth_curve = causal_moving_average(resampled_curve, 3)
            

In [ ]:
# ####get stim algined and rev alogined plots

# '''get reversal features'''

# feature_to_dict = {
#     "body angle mag":(exp_date_to_body_angles,  50, lambda x: np.mean(x) ), 
#     "velocity": (exp_date_to_velocity,   50,  lambda x: np.max(np.abs(x))), 
#     "head angle": (exp_date_to_head_angle,  50,  lambda x: np.mean(x)),#np.arange(0, 6, 0.01)), , 
#     "acceleration": (exp_date_to_acceleration, 50, lambda x: np.max(np.abs(x)))
    
    
# }

# '''get rev start aligned '''
# n_pre_frames = 16*5
# max_plotting_length_frames =  320
# dt = 0.06

# rev_duration_mins = np.arange(0, 20/dt)[::15]#[-5:]
# print(rev_duration_mins)
# rev_diff = 0.5/dt
# rev_duration_maxs = rev_duration_mins+rev_diff

# for rev_duration_min ,rev_duration_max in zip(rev_duration_mins.tolist(),rev_duration_maxs.tolist()):
#     features = [ "acceleration", "velocity", "body angle mag"]
#     all_fig, all_axs = plt.subplots(len(features), 1 )
#     for i, feature in enumerate(features): 
#         exp_date_to_feature, bins, feature_fnc  = feature_to_dict[feature]
#         # for rslds_start_state, rslds_end_state in rslds_pairs: 
#             # '''get number of reversals that: 
#             #     - start in rslds state and end in other rslds state 
#             #     - start in rslds state and stay in rslds state 
#             #     - start in differnt state and end in rslds state 
#             # ...based on this decide which version to plot.. 
#             # '''
#             # print(feature)
#             # # fig, ax = plt.subplots()
#             # feature_vals = []
#             # beh_states = []
#         buffered_event_features = []
#         rev_aligned_feature = align_feature_to_rev_start( exp_date_to_beh, exp_date_to_feature, n_pre_frames =n_pre_frames, 
#                                                          max_plotting_length_frames =  max_plotting_length_frames, 
#                                                          rev_duration_min_frames = rev_duration_min, rev_duration_max_frames = rev_duration_max)
#         # ax1 = axs[i]
#         # print(rev_aligned_feature.shape)
#         # for j in range(2,8):
#         fig, ax1 = plt.subplots()
#         plot_mean_line_graph_with_individ_traces(rev_aligned_feature.T, fps = 1/dt,  fig = fig, ax =ax1 , seconds = None, vline = n_pre_frames*dt, 
#                                                 title = f"{rev_duration_min*dt}", 
#                                                 xlabel ="seconds post-rev start", 
#                                                 ylabel =f"{feature}", color = "blue", ylim =None,  ylabel_font_size = 8, 
#                                                 max_n_tracks = 30,line_width=1.0, 
                                                
                                                
                                            
#                                                 )
        
        
#         plot_mean_line_graph_with_individ_traces(rev_aligned_feature.T, fps = 1/dt,  fig = all_fig, ax =all_axs[i] , seconds = None, vline = n_pre_frames*dt, 
#                                                 title = f"{rev_duration_min*dt}", 
#                                                 xlabel ="seconds post-rev start", 
#                                                 ylabel =f"{feature}", color = "blue", ylim =None,  ylabel_font_size = 8, 
#                                                 max_n_tracks = 30,line_width=1.0
                                                
                                            
#                                                 )



# ################seperate by latency plot aligned to stim staet 

# '''get rev start aligned '''
# n_pre_frames = 16*5
# max_plotting_length_frames =  320
# dt = 0.06

# rev_duration_mins = np.arange(0, 20/dt)[::15]#[-5:]
# print(rev_duration_mins)
# rev_diff = 0.5/dt
# rev_duration_maxs = rev_duration_mins+rev_diff

# for rev_duration_min ,rev_duration_max in zip(rev_duration_mins.tolist(),rev_duration_maxs.tolist()):
#     features = [ "acceleration", "velocity", "body angle mag"]
#     all_fig, all_axs = plt.subplots(len(features), 1 )
#     for i, feature in enumerate(features): 
#         exp_date_to_feature, bins, feature_fnc  = feature_to_dict[feature]
#         # for rslds_start_state, rslds_end_state in rslds_pairs: 
#             # '''get number of reversals that: 
#             #     - start in rslds state and end in other rslds state 
#             #     - start in rslds state and stay in rslds state 
#             #     - start in differnt state and end in rslds state 
#             # ...based on this decide which version to plot.. 
#             # '''
#             # print(feature)
#             # # fig, ax = plt.subplots()
#             # feature_vals = []
#             # beh_states = []
#         buffered_event_features = []
#         rev_aligned_feature = align_feature_to_rev_start( exp_date_to_beh, exp_date_to_feature, n_pre_frames =n_pre_frames, 
#                                                          max_plotting_length_frames =  max_plotting_length_frames, 
#                                                          rev_duration_min_frames = rev_duration_min, rev_duration_max_frames = rev_duration_max)
#         # ax1 = axs[i]
#         # print(rev_aligned_feature.shape)
#         # for j in range(2,8):
#         fig, ax1 = plt.subplots()
#         plot_mean_line_graph_with_individ_traces(rev_aligned_feature.T, fps = 1/dt,  fig = fig, ax =ax1 , seconds = None, vline = n_pre_frames*dt, 
#                                                 title = f"{rev_duration_min*dt}", 
#                                                 xlabel ="seconds post-rev start", 
#                                                 ylabel =f"{feature}", color = "blue", ylim =None,  ylabel_font_size = 8, 
#                                                 max_n_tracks = 30,line_width=1.0, 
                                                
                                                
                                            
#                                                 )
        
        
#         plot_mean_line_graph_with_individ_traces(rev_aligned_feature.T, fps = 1/dt,  fig = all_fig, ax =all_axs[i] , seconds = None, vline = n_pre_frames*dt, 
#                                                 title = f"{rev_duration_min*dt}", 
#                                                 xlabel ="seconds post-rev start", 
#                                                 ylabel =f"{feature}", color = "blue", ylim =None,  ylabel_font_size = 8, 
#                                                 max_n_tracks = 30,line_width=1.0
                                                
                                            
#                                                 )
        

                                    
# ################plto aligned to rev end

# features = ["acceleration", "velocity", "body angle mag",  "head angle"]
# fig, axs = plt.subplots(len(features), 1 )
# for i, feature in enumerate(features): 
#     exp_date_to_feature, bins, feature_fnc  = feature_to_dict[feature]

#     buffered_event_features = []
#     rev_aligned_feature = align_feature_to_rev_end(exp_date_to_beh, exp_date_to_feature,
#                             n_pre_frames = n_pre_frames, max_plotting_length_frames = max_plotting_length_frames, 
#                              rev_duration_min_frames = rev_duration_min, rev_duration_max_frames = rev_duration_max)
    
#     plot_mean_line_graph_with_individ_traces(rev_aligned_feature.T, fps = 1/0.06,  fig = fig, ax = axs[i], seconds = None,title = "", 
#                                              xlabel ="seconds post-rev end", 
#                                              ylabel =f"{feature}", color = "blue", ylim =None,  ylabel_font_size = 8, 
#                                              max_n_tracks = 30,line_width=1.0, vline = n_pre_frames*dt)
                                             
                                             

# ###TODO plot aligned to stim end                            
                                        

In [ ]:
####get stim algined and rev alogined plots

'''get reversal features'''

# feature_to_dict = {
#     "body angle mag":(exp_date_to_body_angles,  50, lambda x: np.mean(x) ), 
#     "velocity": (exp_date_to_velocity,   50,  lambda x: np.max(np.abs(x))), 
#     "head angle": (exp_date_to_head_angle,  50,  lambda x: np.mean(x)),#np.arange(0, 6, 0.01)), , 
#     "acceleration": (exp_date_to_acceleration, 50, lambda x: np.max(np.abs(x)))
    
    
# }
feature_list = [velocity_og, acceleration_og, worm_curvature_og]
feature_labels = ["flav_vel", "flav_accel", "flav_curv"]

dt = 0.6
'''get rev start aligned '''
n_pre_frames = int(6/dt)
max_plotting_length_frames =  320


rev_duration_mins = np.arange(0, 20/dt)
print(rev_duration_mins)
# rev_diff = 0.5/dt
rev_duration_maxs = rev_duration_mins+1

all_durations = []
for i, z_w in enumerate(z):
    # rsldsseq = date_to_discrete_rslds_states[exp_date]
    
    # featureseq = exp_date_to_feature[exp_date]
    state_starts, state_ends = get_state_start_ends(
                                                    1,
                                                    z_w,  
                                                    starts= None, 
                                                    ends = None
                                                    )
    for rev_start_i,rev_end_i in zip( state_starts, state_ends):
        rev_duration_frames = rev_end_i-rev_start_i
        all_durations.append(rev_duration_frames)
        
fig, ax = plt.subplots()
ax.hist(all_durations)
        
        
# for rev_duration_min ,rev_duration_max in zip(rev_duration_mins.tolist(),rev_duration_maxs.tolist()):
#     all_fig, all_axs = plt.subplots(len(feature_list), 1 )
#     for i, (feature, feature_lbl) in enumerate(zip(feature_list, feature_labels)):
#         # exp_date_to_feature, bins, feature_fnc  = feature_to_dict[feature]
#         # for rslds_start_state, rslds_end_state in rslds_pairs: 
#             # '''get number of reversals that: 
#             #     - start in rslds state and end in other rslds state 
#             #     - start in rslds state and stay in rslds state 
#             #     - start in differnt state and end in rslds state 
#             # ...based on this decide which version to plot.. 
#             # '''
#             # print(feature)
#             # # fig, ax = plt.subplots()
#             # feature_vals = []
#             # beh_states = []
#         # buffered_event_features = []
        
#         rev_aligned_feature = align_feature_to_rev_start(z, feature, n_pre_frames = n_pre_frames, 
#                                                          max_plotting_length_frames =  max_plotting_length_frames, 
#                                                          rev_duration_min_frames = rev_duration_min, rev_duration_max_frames = rev_duration_max)
#         fig, ax1 = plt.subplots()
#         plot_mean_line_graph_with_individ_traces(rev_aligned_feature.T, n_pre_frames, fps = 1/dt,  fig = fig, ax =ax1 , seconds = None,# vline = n_pre_frames*dt, 
#                                                 title = f"{np.round(rev_duration_min*dt,2)}", 
#                                                 xlabel ="seconds post-rev start", 
#                                                 ylabel =f"{feature_lbl}", color = "blue", ylim =None,  ylabel_font_size = 8, 
#                                                 max_n_tracks = 30,line_width=1.0, 
                                                
#                                                 )
        
        
#         plot_mean_line_graph_with_individ_traces(rev_aligned_feature.T, n_pre_frames, fps = 1/dt,  fig = all_fig, ax =all_axs[i] , seconds = None,# vline = n_pre_frames*dt, 
#                                                title = f"{np.round(rev_duration_min*dt,2)}", 
#                                                 xlabel ="seconds post-rev start", 
#                                                 ylabel =f"{feature_lbl}", color = "blue", ylim =None,  ylabel_font_size = 8, 
#                                                 max_n_tracks = 30,line_width=1.0
                                                
#                                                 )
#         if i>0:
#             all_axs[i].set_title("")
   


                                    
################plto aligned to rev end

# features = ["acceleration", "velocity", "body angle mag",  "head angle"]
max_plotting_length_frames =  int((20+6)/dt)
n_pre_frames  = int(20/dt)
for rev_duration_min ,rev_duration_max in zip(rev_duration_mins.tolist(),rev_duration_maxs.tolist()):
    all_fig, axs = plt.subplots(len(feature_list), 1 )
    for i, (feature, feature_lbl) in enumerate(zip(feature_list, feature_labels)):
        # exp_date_to_feature, bins, feature_fnc  = feature_to_dict[feature]
        # n_pre_frames  = int(rev_duration_max/dt)+2
        buffered_event_features = []
        rev_aligned_feature = align_feature_to_rev_end(z, feature,
                                n_pre_frames = n_pre_frames, max_plotting_length_frames = max_plotting_length_frames, 
                                rev_duration_min_frames = rev_duration_min, rev_duration_max_frames = rev_duration_max)
        
        plot_mean_line_graph_with_individ_traces(rev_aligned_feature.T, n_pre_frames, fps = 1/dt,  fig = all_fig, ax = axs[i], seconds = None,title = "", 
                                                xlabel ="seconds post-rev end", 
                                                ylabel =f"{feature_lbl}", color = "blue", ylim =None,  ylabel_font_size = 8, 
                                                max_n_tracks = 30,line_width=1.0, vline = n_pre_frames*dt)
        
        fig, ax1 = plt.subplots()
        plot_mean_line_graph_with_individ_traces(rev_aligned_feature.T, n_pre_frames, fps = 1/dt,  fig = fig, ax =ax1 , seconds = None, vline = n_pre_frames*dt, 
                                                    title = f"{np.round(rev_duration_min*dt,2)}", 
                                                    xlabel ="seconds post-rev end", 
                                                    ylabel =f"{feature_lbl}", color = "blue", ylim =None,  ylabel_font_size = 8, 
                                                    max_n_tracks = 30,line_width=1.0, 
                                                    
                                                    )



